In [12]:
import os
import xlwings as xw

def copy_wespac_values(old_file, new_file, output_file, version='3.1'):
    # Define the sheets, columns, and last rows to copy from the old to the new file
    sheets_to_copy = {
        'OF': {'column': 'D', 'last_row': 174},
        'F': {'column': 'D', 'last_row': 345},
        'S': {'column': 'F', 'last_row': 101}
    }

    # Open the old and new WESP-AC files using xlwings
    app = xw.App(visible=False)
    old_wespac = app.books.open(old_file)
    new_wespac = app.books.open(new_file)

    try:
        for sheet, info in sheets_to_copy.items():
            column = info['column']
            last_row = info['last_row']
            
            old_sheet = old_wespac.sheets[sheet]
            new_sheet = new_wespac.sheets[sheet]

            # Unprotect the sheet if it's protected
            try:
                new_sheet.api.Unprotect()
            except Exception as e:
                print(f"Failed to unprotect the sheet '{sheet}': {e}")

            # Handle version 3.1 to 3.4 for each sheet
            if sheet == 'OF':
                # Data shifts and special handling for the OF sheet
                for row in range(5, 109):
                    if row <= 108:
                        # D5 in 3.1 -> D6 in 3.4 (shift by 1 up until D108)
                        old_value = old_sheet.range(f'{column}{row}').value
                        new_sheet.range(f'{column}{row+1}').value = old_value
                    elif row == 109:
                        # D109 in 3.1 is empty, skip
                        continue

                # D110 to D111 in 3.1 -> D110 to D111 in 3.4 (direct mapping)
                new_sheet.range(f'{column}110').value = old_sheet.range(f'{column}110').value

                # D112 in 3.4 corresponds to D111 in 3.1 (shift starts here)
                for row in range(111, 151):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row+1}').value = old_value

                # Special case: from D151-D154 in 3.1 into D149-D150 in 3.4
                for row in range(151, 155, 2):
                    old_value1 = old_sheet.range(f'{column}{row}').value
                    old_value2 = old_sheet.range(f'{column}{row+1}').value
                    new_value = 1 if old_value1 == 1 or old_value2 == 1 else 0
                    new_sheet.range(f'{column}{row-2}').value = new_value  # D149-D150 in 3.4

                # Align from D156 onward until D169 (empty in 3.1)
                for row in range(156, 170):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row-4}').value = old_value

                # D170 to D179 in 3.1 -> D165 to D174 in 3.4
                for row in range(170, 180):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row-5}').value = old_value

            elif sheet == 'F':
                # Data shifts and special handling for the F sheet
                for row in range(5, 124):
                    # D5-D123 in both 3.1 and 3.4 are equal
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row}').value = old_value
                    
                new_sheet.range(f'{column}123').value = 0
                # D123 in 3.1 is D124 in 3.4 (shift starts here)
                for row in range(124, 195):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row+1}').value = old_value
                    
                
                # D193 in 3.1 -> D195 in 3.4 (shift starts here)
                for row in range(193, 222):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row+2}').value = old_value
                
                new_sheet.range(f'{column}230').value = old_sheet.range(f'{column}240').value
                new_sheet.range(f'{column}231').value = old_sheet.range(f'{column}241').value

                # Handling the weird cases from D222 onwards
                # D227 in 3.1 -> D224 in 3.4
                new_sheet.range(f'{column}224').value = old_sheet.range(f'{column}227').value

                # D222-D226 in 3.1 -> D225-D229 in 3.4
                for row in range(222, 227):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row+3}').value = old_value

                # D228-D239 in 3.1 -> D232-D243 in 3.4
                for row in range(228, 240):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row+4}').value = old_value

                # D240 in 3.1 -> D231 in 3.4
                new_sheet.range(f'{column}231').value = old_sheet.range(f'{column}240').value

                # D242-D247 in 3.1 -> D244-D249 in 3.4
                for row in range(242, 248):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row+2}').value = old_value

                # D248 in 3.1 -> D250 in 3.4
                new_sheet.range(f'{column}250').value = old_sheet.range(f'{column}248').value

                # D249-D253 in 3.1 -> D337-D341 in 3.4
                for row in range(249, 254):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row+88}').value = old_value

                # D254 in 3.1 -> D342 in 3.4
                new_sheet.range(f'{column}342').value = old_sheet.range(f'{column}254').value

                # D255 in 3.1 -> D343 in 3.4
                new_sheet.range(f'{column}343').value = old_sheet.range(f'{column}255').value

                # D256-D288 in 3.1 -> D251-D283 in 3.4
                for row in range(256, 289):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row-5}').value = old_value

                # Special case: D289 or D290 in 3.1 -> D284 in 3.4
                old_value1 = old_sheet.range(f'{column}289').value
                old_value2 = old_sheet.range(f'{column}290').value
                new_value = 1 if old_value1 == 1 or old_value2 == 1 else 0
                new_sheet.range(f'{column}284').value = new_value

                # D291-D332 in 3.1 -> D285-D326 in 3.4
                for row in range(291, 333):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row-6}').value = old_value
                
                # D333-D341 in 3.1 -> D328-D336 in 3.4
                for row in range(333, 341):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row-5}').value = old_value

            elif sheet == 'S':
                # The data for S starts at F6 and is the same until F88
                for row in range(6, 89):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row}').value = old_value
                # Leave F89-F101 in 3.4 as is since it extends beyond 3.1

            # Protect the sheet again after modifying
            new_sheet.api.Protect()

        # Save the new WESP-AC file with the copied values
        new_wespac.save(output_file)
        print(f"Values successfully copied from {old_file} to {output_file} (Version: {version})")

    finally:
        # Close both workbooks and quit the app
        old_wespac.close()
        new_wespac.close()
        app.quit()

def process_wespac_folder(version='3.1'):
    # Define the folder name based on version
    folder_name = f'{version}'
    transferred_folder = f'{version}_transferred'
    
    # Create the transferred folder if it doesn't exist
    if not os.path.exists(transferred_folder):
        os.makedirs(transferred_folder)

    # Get a list of all files in the version folder
    input_folder = os.path.join(os.getcwd(), folder_name)  # Assuming folder is in the current directory
    wespac_files = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith('.xlsx')]

    # Define the new template WESP-AC file
    new_wespac_file = 'wespac_3.4.xlsx'

    # Iterate over all WESP-AC files in the folder
    for old_wespac_file in wespac_files:
        # Extract the filename without the extension and add the _3.4 suffix
        base_name = os.path.splitext(os.path.basename(old_wespac_file))[0]
        output_wespac_file = os.path.join(transferred_folder, f'{base_name}_3.4.xlsx')
        
        # Copy values from the old version to the new one
        copy_wespac_values(old_wespac_file, new_wespac_file, output_wespac_file, version=version)

# Define the version you want to process
x_var = '3.1'
process_wespac_folder(version=x_var)


Values successfully copied from C:\Users\marcb\Desktop\wetlands\wespac_transfer\3.1\WESP-AC3.1.xlsx to 3.1_transferred\WESP-AC3.1_3.4.xlsx (Version: 3.1)
